In [1]:
from pymongo import MongoClient
import pandas as pd
from near_funciones import getCompaniesNear, getEmployees, getMoney
import requests
import time
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
client = MongoClient("mongodb://localhost:27017/")

In [3]:
db = client.companies

In [4]:
collection_companies = db.clean_companies

In [5]:
query = collection_companies.find()

In [6]:
data = pd.DataFrame(query)

In [7]:
data.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,description,latitude,longitude,country,state,city,monedas,total_amount_raised,geo
0,5d83a90255488441b3fff55a,Plaxo,web,50,2002.0,None,Contact Management,37.387845,-122.055197,USA,CA,Sunnyvale,Dolares estadounidenses,2.830000e+07,"{'type': 'Point', 'coordinates': [-122.055197,..."
1,5d83a90255488441b3fff55b,Kyte,games_video,40,2006.0,None,Online & Mobile Video Platform,37.788482,-122.409173,USA,CA,San Francisco,Dolares estadounidenses,2.340000e+07,"{'type': 'Point', 'coordinates': [-122.409173,..."
2,5d83a90255488441b3fff55c,Twitter,social,1300,2006.0,None,Real time communication platform,37.776805,-122.416924,USA,CA,San Francisco,Dolares estadounidenses,1.160000e+09,"{'type': 'Point', 'coordinates': [-122.4169244..."
3,5d83a90255488441b3fff55d,eBay,web,15000,1995.0,None,Online Marketplace,37.295005,-121.930035,USA,CA,San Jose,Dolares estadounidenses,6.700000e+06,"{'type': 'Point', 'coordinates': [-121.930035,..."
4,5d83a90255488441b3fff55e,Jajah,mobile,110,2005.0,None,IP Communications Platform,37.423390,-122.089951,USA,CA,Mountain View,Dolares estadounidenses,3.300000e+07,"{'type': 'Point', 'coordinates': [-122.0899512..."


In [8]:
data.shape

(200, 15)

In [9]:
# Creo una nueva columna con el número de empresas que están a un km de distancia
data["num_companies"] = data["geo"].apply(lambda x: getCompaniesNear(collection_companies, x))

In [10]:
# Creo una nueva columna con el número de empleados totales en un radio de un km de distancia

In [11]:
data["num_employees"] = data["geo"].apply(lambda x: getEmployees(collection_companies, x))

In [12]:
# Creo una nueva columna con el dinero total a un kilometro de distancia
data["total_money"] = data["geo"].apply(lambda x: getMoney(collection_companies, x))

In [13]:
data.head(3)

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,description,latitude,longitude,country,state,city,monedas,total_amount_raised,geo,num_companies,num_employees,total_money
0,5d83a90255488441b3fff55a,Plaxo,web,50,2002.0,None,Contact Management,37.387845,-122.055197,USA,CA,Sunnyvale,Dolares estadounidenses,2.830000e+07,"{'type': 'Point', 'coordinates': [-122.055197,...",1,50,2.830000e+07
1,5d83a90255488441b3fff55b,Kyte,games_video,40,2006.0,None,Online & Mobile Video Platform,37.788482,-122.409173,USA,CA,San Francisco,Dolares estadounidenses,2.340000e+07,"{'type': 'Point', 'coordinates': [-122.409173,...",14,1532,5.451700e+08
2,5d83a90255488441b3fff55c,Twitter,social,1300,2006.0,None,Real time communication platform,37.776805,-122.416924,USA,CA,San Francisco,Dolares estadounidenses,1.160000e+09,"{'type': 'Point', 'coordinates': [-122.4169244...",3,1487,1.233400e+09


In [14]:
# San Francisco es la ciudad que más empresas de videojuegos tiene. Voy a tomar como referencia la compañia del 
# sector radicada en esa ciudad que más dinero ha acumulado. Luego extraigo datos de restaurantes veganos,
# bares y cafeterías de la api de Zumato

In [14]:
video_games = data.loc[data["category_code"] == "games_video"]

In [15]:
san_francisco = data.loc[data["city"] == "San Francisco"]
sf_companies = video_games.loc[video_games["city"] == "San Francisco"]
san_francisco

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,description,latitude,longitude,country,state,city,monedas,total_amount_raised,geo,num_companies,num_employees,total_money
1,5d83a90255488441b3fff55b,Kyte,games_video,40,2006.0,None,Online & Mobile Video Platform,37.788482,-122.409173,USA,CA,San Francisco,Dolares estadounidenses,2.340000e+07,"{'type': 'Point', 'coordinates': [-122.409173,...",14,1532,5.451700e+08
2,5d83a90255488441b3fff55c,Twitter,social,1300,2006.0,None,Real time communication platform,37.776805,-122.416924,USA,CA,San Francisco,Dolares estadounidenses,1.160000e+09,"{'type': 'Point', 'coordinates': [-122.4169244...",3,1487,1.233400e+09
9,5d83a90255488441b3fff563,Ustream,games_video,250,2007.0,None,Live Social Video Community,37.392936,-122.079480,USA,CA,San Francisco,Dolares estadounidenses,6.010000e+07,"{'type': 'Point', 'coordinates': [-122.07948, ...",8,1895,4.578300e+08
13,5d83a90255488441b3fff567,Pageflakes,web,20,2006.0,None,AJAX Homepage Service,37.758113,-122.414689,USA,CA,San Francisco,Dolares estadounidenses,4.100000e+06,"{'type': 'Point', 'coordinates': [-122.414689,...",1,20,4.100000e+06
16,5d83a90255488441b3fff56a,Eventbrite,web,200,2006.0,None,Online event management & ticketing,37.779507,-122.390710,USA,CA,San Francisco,Dolares estadounidenses,1.400000e+08,"{'type': 'Point', 'coordinates': [-122.39071, ...",11,1247,3.064000e+08
21,5d83a90255488441b3fff56f,Flixster,web,75,2006.0,None,Movie social networking,37.766909,-122.406676,USA,CA,San Francisco,Dolares estadounidenses,7.180000e+06,"{'type': 'Point', 'coordinates': [-122.406676,...",4,380,9.194800e+08
23,5d83a90255488441b3fff571,PowerReviews,social,120,NaN,None,Social Commerce Solutions,37.779558,-122.393041,USA,CA,San Francisco,Dolares estadounidenses,4.010000e+07,"{'type': 'Point', 'coordinates': [-122.393041,...",12,1290,3.169000e+08
24,5d83a90255488441b3fff572,hi5,games_video,100,2003.0,None,,37.788668,-122.400558,USA,CA,San Francisco,Dolares estadounidenses,5.200000e+07,"{'type': 'Point', 'coordinates': [-122.400558,...",20,1910,7.164700e+08
25,5d83a90255488441b3fff573,GoingOn,software,40,2008.0,None,Private Academic Networks,37.782263,-122.392142,USA,CA,San Francisco,Dolares estadounidenses,8.500000e+06,"{'type': 'Point', 'coordinates': [-122.392142,...",18,1614,4.597000e+08
26,5d83a90255488441b3fff574,Tagged,social,162,2004.0,None,,37.775300,-122.418600,USA,CA,San Francisco,Dolares estadounidenses,2.870000e+07,"{'type': 'Point', 'coordinates': [-122.4186, 3...",3,1487,1.233400e+09


In [21]:
# Oficina de San Francisco con más dinero
sf_office = sf_companies.loc[sf_companies["total_amount_raised"].idxmax()]
sf_office

_id                                             5d83a90255488441b3fff59d
name                                                               Zynga
category_code                                                games_video
number_of_employees                                                  115
founded_year                                                        2007
deadpooled_year                                                     None
description                               Network of gaming applications
latitude                                                         37.7652
longitude                                                       -122.404
country                                                              USA
state                                                                 CA
city                                                       San Francisco
monedas                                          Dolares estadounidenses
total_amount_raised                                

In [32]:
# Coordenadas de la oficina de San Francisco
sf_coord = (sf_office["longitude"], sf_office["latitude"])
sf_coord

(-122.404234, 37.765158)

In [33]:
# Exporto el dataframe entero, además de los correspondiente a San Francisco como un csv

In [34]:
data.to_csv("./output/data.csv", index=False)

In [35]:
san_francisco.to_csv("./output/san_francisco.csv", index=False)

In [37]:
sf_companies.to_csv("./output/sf_companies.csv", index=False)

In [23]:
# Extracción de datos de la Api

In [41]:
zomato_key = os.getenv("zomato_key")

In [42]:
def getVeganRestaurants(lat, lon):
    
    # Hacer una llamada a la api para buscar restaurantes veganos
    # Utilizo un radio de 1 km a la redonda
    # utilizo la clave 308, la cifra que se corresponde con los locales vegetarianos
    
    headers = {
        "user-key": "{}".format(zomato_key)
    }
    url = "https://developers.zomato.com/api/v2.1/search?lat={}&lon={}&radius=1000&cuisines=308".format(lat, lon)
    response = requests.get(url, headers=headers)
    response = response.json()
    return response

In [26]:
# En un principio extraje datos para todas las compaías del dataFrame data
# restaurants = data.apply(lambda x: getVeganRestaurants(x["latitude"], x["longitude"]), axis=1)

In [ ]:
# Extracción de los restaurantes veganos en un rdio de 1 km respecto a la oficina tomda como referencia

In [50]:
lon, lat = sf_coord
sf_restaurants = getVeganRestaurants(lat, lon)

In [46]:
def getInfoRestaurants(data):
    
    # función para extraer la info del json que devuelve la api
    
    info_restaurants = []
    for res in data["restaurants"]:
        res_dict = {
                   "name": res["restaurant"]["name"],
                    "address": res["restaurant"]["location"]["address"],
                   "lat": res["restaurant"]["location"]["latitude"],
                   "lon": res["restaurant"]["location"]["longitude"],
                   "timings": res["restaurant"]["timings"]
               }
        info_restaurants.append(res_dict)
    return info_restaurants

In [51]:
data_sfrest = pd.DataFrame(getInfoRestaurants(sf_restaurants))
data_sfrest.head(3)

,name,address,lat,lon,timings
0,Greens,"2 Marina Boulevard, San Francisco 94123",37.8067444444,-122.4321444444,"5:30 PM to 9 PM (Mon), 11:30 AM to 2:30 PM, 5:..."
1,Gracias Madre,"2211 Mission District Street, San Francisco 94110",37.7615500000,-122.4195555556,"11 AM to 10 PM (Mon-Tue), 11 AM to 11 PM (Wed-..."
2,Udupi Palace,"1007 Valencia Street, San Francisco 94110",37.7567138889,-122.4211666667,"11:30 AM to 10 PM (Mon, Tue, Wed, Thu, Sun), 1..."


In [52]:
data_sfrest.to_csv("./output/vegan_restaurantssf.csv", index=False)

In [53]:
# llamada api para buscar bares

In [54]:
def getBares(lat, lon):
    # Hacer una llamada a la api para buscar bares
    # Utilizo un radio de 1 km a la redonda
    # utilizo la clave 3, la cifra que se corresponde con los locales de ocio nocturno
    
    headers = {
        "user-key": "{}".format(zomato_key)
    }
    url = "https://developers.zomato.com/api/v2.1/search?entity_type=zone&lat={}&lon={}&radius=1000&category=3".format(lat, lon)
    response = requests.get(url, headers=headers)
    response = response.json()
    return response

In [55]:
lon, lat = sf_coord
sf_bares = getBares(lat, lon)

In [56]:
def getInfoBares(data):
    
    # función para extraer la info del json que devuelve la api
    
    info_bares = []
    for bar in data["restaurants"]:
        bares_dict = {
                   "name": bar["restaurant"]["name"],
                    "address": bar["restaurant"]["location"]["address"],
                   "lat": bar["restaurant"]["location"]["latitude"],
                   "lon": bar["restaurant"]["location"]["longitude"],
                   "timings": bar["restaurant"]["timings"]
               }
        info_bares.append(bares_dict)
    return info_bares

In [57]:
data_sfbares = pd.DataFrame(getInfoBares(sf_bares))

In [59]:
data_sfbares.head(3)

,name,address,lat,lon,timings
0,Foreign Cinema,"2534 Mission District Street, San Francisco 94110",37.7565305556,-122.4190444444,"5:30 PM to 10 PM (Mon-Wed),5:30 PM to 11 PM (T..."
1,Gary Danko,"800 North Point Street, San Francisco 94109",37.8057055556,-122.4205638889,5 PM to 12 Midnight (Mon-Sun)
2,Nopa,"560 Divisadero Street, San Francisco 94117",37.7748527778,-122.4378805556,"5 PM to 1 AM (Mon-Fri),11 AM to 2:30 PM, 5 PM ..."


In [60]:
data_sfbares.to_csv("./output/baressf.csv", index=False)

In [61]:
# llamada api para buscar cafés

In [62]:
def getCafes(lat, lon):
    # Hacer una llamada a la api para buscar cafes
    # Utilizo un radio de 1 km a la redonda
    # Utilizo la clave 6, la cifra que se corresponde con los cafes
    
    headers = {
        "user-key": "{}".format(zomato_key)
    }
    url = "https://developers.zomato.com/api/v2.1/search?entity_type=zone&lat={}&lon={}&radius=1000&category=6".format(lat, lon)
    response = requests.get(url, headers=headers)
    response = response.json()
    return response

In [ ]:
# cafes = data.apply(lambda x: getCafes(x["latitude"], x["longitude"]), axis=1)

In [63]:
lon, lat = sf_coord
sf_cafes = getCafes(lat, lon)

In [64]:
def getInfoCafes(data):
    
    # función para extraer la info del json que devuelve la api
    
    info_cafes = []
    for caf in data["restaurants"]:
        cafes_dict = {
            "name": caf["restaurant"]["name"],
            "address": caf["restaurant"]["location"]["address"],
            "lat": caf["restaurant"]["location"]["latitude"],
            "lon": caf["restaurant"]["location"]["longitude"],
            "timings": caf["restaurant"]["timings"]
        }
        info_cafes.append(cafes_dict)
    return info_cafes


In [65]:
data_sfcafes = pd.DataFrame(getInfoCafes(sf_cafes))

In [66]:
data_sfcafes.head(3)

,name,address,lat,lon,timings
0,Tartine Bakery,"600 Guerrero Street, San Francisco 94110",37.7614861111,-122.4239500000,"8 AM to 7 PM (Mon), 7:30 AM to 7 PM (Tue-Wed),..."
1,Boudin Bakery,"160 Jefferson Street, Lower Level, San Francis...",37.8087800000,-122.4146440000,"8 AM to 9 PM (Mon-Fri, Sun), 8 AM to 10 PM (Sat)"
2,Boudin Bakery & Cafe,"Pier 39, San Francisco 94133",37.8088722222,-122.4100027778,8 AM to 11 PM (Mon-Sun)


In [67]:
data_sfcafes.to_csv("./output/cafessf.csv", index=False)

In [ ]:
# Visualizaciones

In [70]:
import folium

In [117]:
# Utilizo como punto de referencia la empresa seleccionada tiles="Stamen Toner"

sf_map = folium.Map(location=[37.765158, -122.404234], tiles="Stamen Toner", zoom_start=14)

folium.Circle(
    radius=100,
    location=[37.765158, -122.404234],
    popup='Office reference',
    color='crimson',
    fill=True,
).add_to(sf_map)

# Añado las empresas que hay en San Francisco

for lat, lon in list(zip(san_francisco["latitude"], san_francisco["longitude"])):
    folium.Circle([lat, lon], radius=50, popup="Company", color="blue", fill=True).add_to(sf_map)
    
# Añado los restaurntes veganos

for lat, lon in list(zip(data_sfrest["lat"], data_sfrest["lon"])):
    folium.Circle([lat, lon], radius=50, popup="Vegan restaurant", color="green", fill=True).add_to(sf_map)


# Añado las cafeterías

for lat, lon in list(zip(data_sfcafes["lat"], data_sfcafes["lon"])):
    folium.Circle([lat, lon], radius=50, popup="Cafe", color="yellow", fill=True).add_to(sf_map)
    

# Añado los bares

for lat, lon in list(zip(data_sfbares["lat"], data_sfbares["lon"])):
    folium.Circle([lat, lon], radius=50, popup="Bar", color="pink", fill=True).add_to(sf_map)

In [118]:
sf_map #icon=folium.Icon(color='red', icon='info-sign')

In [119]:
sf_map.save('./output/index.html')